In [1]:
from huggingface_hub import notebook_login
import os
#os.environ["CUDA_LAUNCH_BLOCKING"]="1"
os.environ["TOKENIZERS_PARALLELISM"]="true"
notebook_login()
token = "huggingface-key"

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from tqdm import tqdm
from utils import *
import gc


model_name = "mistralai/Mistral-7B-v0.3"
model, tokenizer, lm_head = load_model_and_tokenizer(model_name, type = "lora")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Total trainable parameters after: 9,437,184


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Perplexity: 5.44


In [3]:
torch.cuda.empty_cache()
_ = gc.collect()

In [4]:
# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=lr)
num_epochs = 3
num_training_steps = num_epochs * len(mrpc_dataset["train"])
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=num_training_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
# Convert to PyTorch datasets
train_dataset = mrpc_dataset["train"]
batch_size = 8

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    #for batch in tqdm(train_dataset, desc=f"Epoch {epoch + 1}"):
    #for i in tqdm(range(0, len(train_dataset), batch_size)):
    for i in tqdm(range(0, 1000, batch_size)):
        batch = train_dataset[i:i+batch_size]
        inputs = tokenizer(batch["sentence1"], batch["sentence2"], return_tensors="pt",
                           truncation=True, padding = True, max_length = max_len).to(device)
        optimizer.zero_grad()
        outputs = model(**inputs)["logits"][:, -1, :]#.cpu()
        #loss = criterion(outputs, torch.LongTensor(batch["label"]).to("cuda"))
        y = torch.LongTensor(batch["label"]).to(device)
        loss = torch.nn.functional.cross_entropy(outputs, y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        #outputs.detach().cpu()
        #y.detach().cpu()
        #inputs["input_ids"].detach().cpu()
        #inputs["attention_mask"].detach().cpu()
        torch.cuda.empty_cache()
        _ = gc.collect()

    print(f"Epoch {epoch + 1}, Average loss: {total_loss / len(train_dataset)}")

100%|██████████| 125/125 [02:37<00:00,  1.26s/it]


Epoch 1, Average loss: 0.053962597515494454


100%|██████████| 125/125 [02:37<00:00,  1.26s/it]


Epoch 2, Average loss: 0.02225494865204168


100%|██████████| 125/125 [02:37<00:00,  1.26s/it]

Epoch 3, Average loss: 0.016627136381767535


In [6]:
print("Evaluating on MRPC...")
mrpc_accuracy = evaluate_mrpc(model, tokenizer, mrpc_dataset, device)
print(f"MRPC Accuracy: {mrpc_accuracy:.4f}")

Evaluating on MRPC...


100%|██████████| 408/408 [00:28<00:00, 14.22it/s]

MRPC Accuracy: 0.6495


In [7]:
mrpc_head = model.lm_head

In [8]:
torch.manual_seed(1311)

num_classes = 2
model.lm_head = torch.nn.Linear(in_features=4096, out_features=num_classes).to(device)

In [9]:
# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=lr)
num_epochs = 3
num_training_steps = num_epochs * len(sst_dataset["train"])
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=num_training_steps)

In [10]:
# Convert to PyTorch datasets
train_dataset = sst_dataset["train"]
batch_size = 8

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    #for batch in tqdm(train_dataset, desc=f"Epoch {epoch + 1}"):
    #for i in tqdm(range(0, len(train_dataset), batch_size)):
    for i in tqdm(range(0, 1000, batch_size)):
        batch = train_dataset[i:i+batch_size]
        inputs = tokenizer(batch["sentence"], return_tensors="pt",
                           truncation=True, padding = True, max_length = max_len).to(device)
        optimizer.zero_grad()
        outputs = model(**inputs)["logits"][:, -1, :]#.cpu()
        #loss = criterion(outputs, torch.LongTensor(batch["label"]).to("cuda"))
        y = torch.LongTensor(batch["label"]).to(device)
        loss = torch.nn.functional.cross_entropy(outputs, y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        #outputs.detach().cpu()
        #y.detach().cpu()
        #inputs["input_ids"].detach().cpu()
        #inputs["attention_mask"].detach().cpu()
        torch.cuda.empty_cache()
        _ = gc.collect()

    print(f"Epoch {epoch + 1}, Average loss: {total_loss / len(train_dataset)}")

100%|██████████| 125/125 [01:12<00:00,  1.72it/s]


Epoch 1, Average loss: 0.0015035631459601113


100%|██████████| 125/125 [01:12<00:00,  1.74it/s]


Epoch 2, Average loss: 0.00041665321909103385


100%|██████████| 125/125 [01:10<00:00,  1.77it/s]

Epoch 3, Average loss: 0.001446018162814781


In [11]:
print("Evaluating on SST-2...")
accuracy = evaluate_sst2(model, tokenizer, sst_dataset, device)
print(f"SST-2 Accuracy: {accuracy:.4f}")

Evaluating on SST-2...


100%|██████████| 872/872 [00:47<00:00, 18.24it/s]

SST-2 Accuracy: 0.5092


In [12]:
torch.cuda.empty_cache()
_ = gc.collect()

In [13]:
sst_head = model.lm_head
model.lm_head = mrpc_head

In [14]:
print("Evaluating on MRPC...")
mrpc_accuracy = evaluate_mrpc(model, tokenizer, mrpc_dataset, device)
print(f"MRPC Accuracy: {mrpc_accuracy:.4f}")

Evaluating on MRPC...


100%|██████████| 408/408 [00:28<00:00, 14.17it/s]

MRPC Accuracy: 0.3162


In [15]:
torch.manual_seed(1311)

num_classes = 3
model.lm_head = torch.nn.Linear(in_features=4096, out_features=num_classes).to(device)

In [16]:
# Set up optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=lr)
num_epochs = 3
num_training_steps = num_epochs * len(sentiment_dataset["test"])
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=num_training_steps)

In [17]:
# Convert to PyTorch datasets
train_dataset = sentiment_dataset["test"]
batch_size = 8

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    #for batch in tqdm(train_dataset, desc=f"Epoch {epoch + 1}"):
    #for i in tqdm(range(0, len(train_dataset), batch_size)):
    for i in tqdm(range(0, len(train_dataset), batch_size)):
        batch = train_dataset[i:i+batch_size]
        inputs = tokenizer(batch["text"], return_tensors="pt",
                           truncation=True, padding = True, max_length = max_len).to(device)
        optimizer.zero_grad()
        outputs = model(**inputs)["logits"][:, -1, :]#.cpu()
        #loss = criterion(outputs, torch.LongTensor(batch["label"]).to("cuda"))
        y = torch.LongTensor(batch["sentiment"]).to(device) // 4
        loss = torch.nn.functional.cross_entropy(outputs, y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        #outputs.detach().cpu()
        #y.detach().cpu()
        #inputs["input_ids"].detach().cpu()
        #inputs["attention_mask"].detach().cpu()
        torch.cuda.empty_cache()
        _ = gc.collect()

    print(f"Epoch {epoch + 1}, Average loss: {total_loss / len(train_dataset)}")

100%|██████████| 63/63 [00:45<00:00,  1.38it/s]


Epoch 1, Average loss: 0.11045337237986216


100%|██████████| 63/63 [00:44<00:00,  1.40it/s]


Epoch 2, Average loss: 0.10219422069539028


100%|██████████| 63/63 [00:43<00:00,  1.44it/s]

Epoch 3, Average loss: 0.09562826596469765


In [18]:
print("Evaluating on Sentiment140...")
sentiment_accuracy = evaluate_sentiment(model, tokenizer, sentiment_dataset, device)
print(f"Sentiment Accuracy: {sentiment_accuracy:.4f}")

Evaluating on Sentiment140...


100%|██████████| 8/8 [00:34<00:00,  4.35s/it]

Sentiment Accuracy: 1.0000


In [19]:
sentiment_head = model.lm_head
sentiment_head

Linear(in_features=4096, out_features=3, bias=True)

In [20]:
model.lm_head = mrpc_head
print("Evaluating on MRPC...")
mrpc_accuracy = evaluate_mrpc(model, tokenizer, mrpc_dataset, device)
print(f"MRPC Accuracy: {mrpc_accuracy:.4f}")

Evaluating on MRPC...


100%|██████████| 408/408 [00:28<00:00, 14.18it/s]

MRPC Accuracy: 0.3162


In [21]:
model.lm_head = sst_head
print("Evaluating on SST-2...")
accuracy = evaluate_sst2(model, tokenizer, sst_dataset, device)
print(f"SST-2 Accuracy: {accuracy:.4f}")

Evaluating on SST-2...


100%|██████████| 872/872 [00:47<00:00, 18.32it/s]

SST-2 Accuracy: 0.4908


In [22]:
torch.cuda.empty_cache()
_ = gc.collect()

In [23]:
model.lm_head = lm_head
ppl = calculate_perplexity(model, tokenizer, txt)
print("Perplexity:", round(ppl, 2))

Perplexity: 65055.02


In [24]:
torch.cuda.empty_cache()
_ = gc.collect()

In [25]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): LinearWithLoRA(
            (linear): Linear(in_features=4096, out_features=4096, bias=False)
            (lora): LoRALayer()
          )
          (k_proj): LinearWithLoRA(
            (linear): Linear(in_features=4096, out_features=1024, bias=False)
            (lora): LoRALayer()
          )
          (v_proj): LinearWithLoRA(
            (linear): Linear(in_features=4096, out_features=1024, bias=False)
            (lora): LoRALayer()
          )
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
         